In [33]:
# Cell 1: 環境設定とセットアップ
"""
LA-Bench 2025: 実験手順生成タスク
Baseline Implementation for Google Colaboratory
GitHub: https://github.com/lasa-or-jp/la-bench.git
"""

#@title 1. 環境セットアップ { display-mode: "form" }
#@markdown このセルを実行して必要なライブラリをインストールし、リポジトリをクローンします。


import os
import sys
from pathlib import Path

# Colabかどうかの確認
try:
    import google.colab
    IN_COLAB = True
    print("✅ Google Colaboratory環境を検出しました")
except ImportError:
    IN_COLAB = False
    print("⚠️ ローカル環境で実行中です")

# 必要なライブラリのインストール
print("\n📦 必要なライブラリをインストール中...")
!pip install -q openai tenacity pyyaml tqdm python-Levenshtein pandas numpy matplotlib seaborn

print("✅ ライブラリのインストール完了")

# GitHubリポジトリのクローン
REPO_URL = "https://github.com/lasa-or-jp/la-bench.git"
REPO_NAME = "la-bench"

if not os.path.exists(REPO_NAME):
    print(f"\n📥 リポジトリをクローン中: {REPO_URL}")
    !git clone -q {REPO_URL}
    print(f"✅ リポジトリのクローン完了: {REPO_NAME}/")
else:
    print(f"\n📂 リポジトリは既に存在します: {REPO_NAME}/")
    print("📥 最新版に更新中...")
    !cd {REPO_NAME} && git pull -q
    print("✅ 更新完了")

# 作業ディレクトリの設定
WORK_DIR = Path(REPO_NAME)
os.chdir(WORK_DIR)
print(f"\n📍 作業ディレクトリ: {os.getcwd()}")

# ディレクトリ構造の確認
print("\n📊 プロジェクト構造:")
!ls -la

✅ Google Colaboratory環境を検出しました

📦 必要なライブラリをインストール中...
✅ ライブラリのインストール完了

📥 リポジトリをクローン中: https://github.com/lasa-or-jp/la-bench.git
✅ リポジトリのクローン完了: la-bench/

📍 作業ディレクトリ: /content/la-bench/la-bench/la-bench

📊 プロジェクト構造:
total 52
drwxr-xr-x  8 root root 4096 Aug 10 09:36 .
drwxr-xr-x 10 root root 4096 Aug 10 09:36 ..
drwxr-xr-x  4 root root 4096 Aug 10 09:36 code
-rw-r--r--  1 root root 2658 Aug 10 09:36 CONTRIBUTING.md
drwxr-xr-x  4 root root 4096 Aug 10 09:36 data
drwxr-xr-x  2 root root 4096 Aug 10 09:36 docs
drwxr-xr-x  8 root root 4096 Aug 10 09:36 .git
drwxr-xr-x  3 root root 4096 Aug 10 09:36 .github
-rw-r--r--  1 root root  971 Aug 10 09:36 .gitignore
-rw-r--r--  1 root root 1101 Aug 10 09:36 LICENSE
-rw-r--r--  1 root root 4196 Aug 10 09:36 README.md
drwxr-xr-x  2 root root 4096 Aug 10 09:36 submissions


In [34]:
# Cell 2: OpenAI APIキーの設定
#@title 2. OpenAI API Key設定 { display-mode: "form" }
#@markdown OpenAI APIキーを入力してください。キーは安全に管理されます。

import getpass
from google.colab import userdata

# APIキーの取得方法を選択
use_secrets = True  #@param {type:"boolean"}
#@markdown ☝️ Google Colab Secretsを使用する場合はチェック

if IN_COLAB:
    if use_secrets:
        try:
            # Colab Secretsから取得
            API_KEY = userdata.get('OPENAI_API_KEY')
            print("✅ APIキーをSecretsから取得しました")
        except Exception as e:
            print("⚠️ Secretsからの取得に失敗しました")
            print("左側のパネルの🔑アイコンから'OPENAI_API_KEY'を設定してください")
            API_KEY = None
    else:
        # 直接入力
        api_key_input = getpass.getpass("🔑 OpenAI API Keyを入力: ")
        if api_key_input:
            API_KEY = api_key_input
            os.environ['OPENAI_API_KEY'] = API_KEY
            print("✅ APIキーが設定されました")
        else:
            API_KEY = None
            print("⚠️ APIキーが設定されていません（ヒューリスティック手法のみ使用）")
else:
    # ローカル環境の場合
    API_KEY = os.getenv("OPENAI_API_KEY")
    if not API_KEY:
        API_KEY = input("OpenAI API Key: ")

# APIキーの検証
if API_KEY:
    print(f"🔑 APIキー: {'*' * 20}{API_KEY[-4:]}")
else:
    print("⚠️ GPT機能は使用できません")

✅ APIキーをSecretsから取得しました
🔑 APIキー: ********************JacA


In [35]:
# Cell 3: ライブラリのインポートと設定
#@title 3. ライブラリのインポート { display-mode: "form" }

import json
import yaml
import re
import time
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Any
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

# データ処理
import pandas as pd
import numpy as np
from dataclasses import dataclass, field, asdict

# OpenAI API
try:
    from openai import OpenAI
    OPENAI_AVAILABLE = True
except ImportError:
    OPENAI_AVAILABLE = False
    print("⚠️ OpenAIライブラリが利用できません")

from tenacity import retry, stop_after_attempt, wait_exponential

# 評価メトリクス
from difflib import SequenceMatcher
try:
    import Levenshtein
    LEVENSHTEIN_AVAILABLE = True
except ImportError:
    LEVENSHTEIN_AVAILABLE = False

# プログレスバー (Colab対応)
from tqdm.auto import tqdm

# 可視化
import matplotlib.pyplot as plt
import seaborn as sns
if IN_COLAB:
    # Colabでの日本語フォント設定
    !apt-get -q install fonts-noto-cjk > /dev/null 2>&1
    import matplotlib
    import matplotlib.font_manager as fm

    # フォントキャッシュをクリアして再構築
    # fontManagerのフォントリストをクリア
    fm.fontManager.ttflist.clear()
    fm.fontManager.afmlist.clear()

    # fontManagerを再初期化（新しいフォントを検出）
    fm.fontManager = fm.FontManager()

    # 利用可能なフォントを確認
    available_fonts = [f.name for f in fm.fontManager.ttflist]
    japanese_fonts = [f for f in available_fonts if 'CJK' in f or 'Noto' in f]

    if japanese_fonts:
        print(f"✅ 日本語フォントを検出: {japanese_fonts[:3]}...")  # 最初の3つだけ表示

        # 日本語フォントの設定
        plt.rcParams['font.family'] = 'sans-serif'
        plt.rcParams['font.sans-serif'] = ['Noto Sans CJK JP'] + plt.rcParams['font.sans-serif']

        # マイナス記号の表示設定
        plt.rcParams['axes.unicode_minus'] = False

        print("✅ 日本語フォントを設定しました")
    else:
        print("⚠️ 日本語フォントが見つかりません。グラフの日本語が文字化けする可能性があります")

# ログ設定
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger(__name__)

print("="*60)
print("LA-Bench 2025 Baseline Implementation")
print(f"実行環境: {'Google Colab' if IN_COLAB else 'Local'}")
print(f"実行時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"OpenAI利用可能: {OPENAI_AVAILABLE and API_KEY is not None}")
print("="*60)

✅ 日本語フォントを検出: ['Noto Sans CJK JP', 'Noto Serif CJK JP', 'Noto Sans CJK JP']...
✅ 日本語フォントを設定しました
LA-Bench 2025 Baseline Implementation
実行環境: Google Colab
実行時刻: 2025-08-10 09:36:41
OpenAI利用可能: True


In [36]:
# Cell 4: データ構造とユーティリティ
#@title 4. データ構造の定義 { display-mode: "form" }

@dataclass
class ProtocolStep:
    """プロトコルのステップ"""
    id: int
    text: str

@dataclass
class ResourceInfo:
    """リソース情報（エイリアス、マッピング、制約）"""
    aliases: Dict[str, str] = field(default_factory=dict)
    mappings: Dict[str, str] = field(default_factory=dict)
    constraints: Dict[str, Any] = field(default_factory=dict)

@dataclass
class ExperimentTask:
    """実験タスクのデータ構造"""
    id: str
    instruction: str
    protocol: List[ProtocolStep]
    resources: ResourceInfo
    ground_truth: Optional[List[ProtocolStep]] = None

    @classmethod
    def from_json(cls, json_data: Dict) -> 'ExperimentTask':
        """JSONデータからExperimentTaskを生成"""
        protocol_steps = [
            ProtocolStep(id=step['id'], text=step['text'])
            for step in json_data['protocol']['steps']
        ]

        resources = ResourceInfo(
            aliases=json_data['resources'].get('aliases', {}),
            mappings=json_data['resources'].get('mappings', {}),
            constraints=json_data['resources'].get('constraints', {})
        )

        ground_truth = None
        if 'ground_truth' in json_data and json_data['ground_truth']:
            ground_truth = [
                ProtocolStep(id=step['id'], text=step['text'])
                for step in json_data['ground_truth']['steps']
            ]

        return cls(
            id=json_data['id'],
            instruction=json_data['instruction'],
            protocol=protocol_steps,
            resources=resources,
            ground_truth=ground_truth
        )

    @classmethod
    def from_yaml(cls, yaml_data: Dict) -> 'ExperimentTask':
        """YAMLデータからExperimentTaskを生成"""
        return cls.from_json(yaml_data)

print("✅ データ構造を定義しました")

✅ データ構造を定義しました


In [37]:
# Cell 5: データローダー（Colab対応）
#@title 5. データローダーの実装 { display-mode: "form" }

class DataLoader:
    """実験タスクデータのローダー"""

    def __init__(self, data_dir: str = "./data"):
        self.data_dir = Path(data_dir)
        if not self.data_dir.exists():
            logger.warning(f"データディレクトリが存在しません: {self.data_dir}")
            self.data_dir.mkdir(parents=True, exist_ok=True)

    def check_data_availability(self) -> Dict[str, bool]:
        """データの利用可能性をチェック"""
        availability = {
            'example_json': False,
            'example_yaml': False,
            'public': False,
            'private': False
        }

        if (self.data_dir / "example" / "json").exists():
            json_files = list((self.data_dir / "example" / "json").glob("task_*.json"))
            availability['example_json'] = len(json_files) > 0

        if (self.data_dir / "example" / "yaml").exists():
            yaml_files = list((self.data_dir / "example" / "yaml").glob("task_*.yaml"))
            availability['example_yaml'] = len(yaml_files) > 0

        if (self.data_dir / "public").exists():
            public_files = list((self.data_dir / "public").glob("pub_task_*.yaml"))
            availability['public'] = len(public_files) > 0

        if (self.data_dir / "private").exists():
            private_files = list((self.data_dir / "private").glob("pri_task_*.yaml"))
            availability['private'] = len(private_files) > 0

        return availability

    def load_task(self, file_path: str) -> ExperimentTask:
        """単一タスクファイルの読み込み"""
        file_path = Path(file_path)

        if not file_path.exists():
            raise FileNotFoundError(f"タスクファイルが見つかりません: {file_path}")

        if file_path.suffix == '.json':
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            return ExperimentTask.from_json(data)
        elif file_path.suffix == '.yaml':
            with open(file_path, 'r', encoding='utf-8') as f:
                data = yaml.safe_load(f)
            return ExperimentTask.from_yaml(data)
        else:
            raise ValueError(f"サポートされていないファイル形式: {file_path.suffix}")

    def load_dataset(self, dataset_type: str = "example") -> List[ExperimentTask]:
        """データセット全体の読み込み"""
        tasks = []

        if dataset_type == "example":
            # JSONとYAMLの両方をチェック
            json_dir = self.data_dir / "example" / "json"
            yaml_dir = self.data_dir / "example" / "yaml"

            # JSON優先で読み込み
            if json_dir.exists():
                for file_path in sorted(json_dir.glob("task_*.json")):
                    try:
                        tasks.append(self.load_task(file_path))
                    except Exception as e:
                        logger.error(f"タスク読み込みエラー {file_path}: {e}")
            elif yaml_dir.exists():
                for file_path in sorted(yaml_dir.glob("task_*.yaml")):
                    try:
                        tasks.append(self.load_task(file_path))
                    except Exception as e:
                        logger.error(f"タスク読み込みエラー {file_path}: {e}")

        elif dataset_type == "public":
            public_dir = self.data_dir / "public"
            if public_dir.exists():
                for file_path in sorted(public_dir.glob("pub_task_*.yaml")):
                    try:
                        tasks.append(self.load_task(file_path))
                    except Exception as e:
                        logger.error(f"タスク読み込みエラー {file_path}: {e}")

        elif dataset_type == "private":
            private_dir = self.data_dir / "private"
            if private_dir.exists():
                for file_path in sorted(private_dir.glob("pri_task_*.yaml")):
                    try:
                        tasks.append(self.load_task(file_path))
                    except Exception as e:
                        logger.error(f"タスク読み込みエラー {file_path}: {e}")

        logger.info(f"{dataset_type}データセットから{len(tasks)}個のタスクを読み込みました")
        return tasks

# データ利用可能性のチェック
loader = DataLoader()
availability = loader.check_data_availability()
print("📊 データ利用可能性:")
for key, available in availability.items():
    status = "✅" if available else "❌"
    print(f"  {status} {key}")

📊 データ利用可能性:
  ❌ example_json
  ❌ example_yaml
  ❌ public
  ❌ private


In [38]:
# Cell 6: サンプルデータの作成（データがない場合）
#@title 6. サンプルデータの作成 { display-mode: "form" }
#@markdown データが存在しない場合、サンプルデータを作成します

create_sample = True #@param {type:"boolean"}

if create_sample and not any(availability.values()):
    print("📝 サンプルデータを作成中...")

    # ディレクトリの作成
    sample_dir = Path("data/example/json")
    sample_dir.mkdir(parents=True, exist_ok=True)

    # サンプルタスク1
    sample_task_1 = {
        "id": "0001",
        "instruction": "T75フラスコに線維芽細胞を1本起こしてください",
        "protocol": {
            "steps": [
                {"id": 1, "text": "15 mL遠沈管に9 mL、Dishに10 mL血清入り培地を入れて恒温槽、インキュベーターで温める"},
                {"id": 2, "text": "液体窒素タンクから起こす細胞を素早く取り出す"},
                {"id": 3, "text": "クライオチューブのキャップを少し緩め、再度閉める"},
                {"id": 4, "text": "恒温槽でクライオチューブを温め、氷の塊がやや残るぐらいまで溶かす"},
                {"id": 5, "text": "9 mLの温めた培地を一部移し、溶けた部分から遠沈管に戻す"},
                {"id": 6, "text": "遠沈管の蓋をしっかり閉め、軽く転倒混和する"},
                {"id": 7, "text": "1000 rpm、5 min、室温で遠心する"},
                {"id": 8, "text": "上清を取り除き、Dishで温めた培地で懸濁する"},
                {"id": 9, "text": "細胞懸濁液を培養器に播種する"},
                {"id": 10, "text": "播種ムラができないように培養器を揺らし、静かにインキュベーターに入れる"}
            ]
        },
        "resources": {
            "aliases": {"Dish": "培養器"},
            "mappings": {"Dish": "T75フラスコ"},
            "constraints": {
                "optional_steps": [
                    {"name": "恒温槽の電源を消す", "earliest_index": 7}
                ]
            }
        },
        "ground_truth": {
            "steps": [
                {"id": 1, "text": "恒温槽の電源を入れる"},
                {"id": 2, "text": "15 mL遠沈管に9 mL、T75フラスコに15 mL血清入り培地を入れる"},
                {"id": 3, "text": "遠沈管は恒温槽、T75フラスコはインキュベーターに入れる"},
                {"id": 4, "text": "液体窒素タンクから線維芽細胞のクライオチューブを取り出す"},
                {"id": 5, "text": "クライオチューブのキャップを少し緩め、再度閉める"},
                {"id": 6, "text": "恒温槽でクライオチューブを温め、氷の塊がやや残るぐらいまで溶かす"},
                {"id": 7, "text": "遠沈管から、9 mLの温めた培地を一部移し、溶けた部分から遠沈管に戻す"},
                {"id": 8, "text": "クライオチューブに培地を加えてピペッティングして、細胞を遠沈管に回収する"},
                {"id": 9, "text": "遠沈管の蓋をしっかり閉め、軽く転倒混和する"},
                {"id": 10, "text": "遠沈管を1000 rpm、5 min、室温で遠心する"},
                {"id": 11, "text": "上清を取り除き、T75フラスコで温めた培地で懸濁する"},
                {"id": 12, "text": "細胞懸濁液をT75フラスコに戻す"},
                {"id": 13, "text": "播種ムラができないようにT75フラスコを揺らし、静かにインキュベーターに入れる"},
                {"id": 14, "text": "恒温槽の電源を消す"}
            ]
        }
    }

    # サンプルタスク2
    sample_task_2 = {
        "id": "0002",
        "instruction": "24ウェルプレートで細胞の培地交換を行ってください",
        "protocol": {
            "steps": [
                {"id": 1, "text": "培地を恒温槽で温める"},
                {"id": 2, "text": "プレートをインキュベーターから取り出す"},
                {"id": 3, "text": "古い培地をアスピレーターで除去する"},
                {"id": 4, "text": "新しい培地を各ウェルに加える"},
                {"id": 5, "text": "プレートをインキュベーターに戻す"}
            ]
        },
        "resources": {
            "aliases": {"プレート": "培養プレート"},
            "mappings": {"プレート": "24ウェルプレート", "培地": "DMEM培地"},
            "constraints": {}
        },
        "ground_truth": {
            "steps": [
                {"id": 1, "text": "DMEM培地を恒温槽で37°Cに温める"},
                {"id": 2, "text": "24ウェルプレートをインキュベーターから取り出す"},
                {"id": 3, "text": "古い培地をアスピレーターで各ウェルから慎重に除去する"},
                {"id": 4, "text": "新しいDMEM培地を各ウェルに1 mL加える"},
                {"id": 5, "text": "24ウェルプレートをインキュベーターに戻す"}
            ]
        }
    }

    # ファイルの保存
    with open(sample_dir / "task_0001.json", 'w', encoding='utf-8') as f:
        json.dump(sample_task_1, f, ensure_ascii=False, indent=2)

    with open(sample_dir / "task_0002.json", 'w', encoding='utf-8') as f:
        json.dump(sample_task_2, f, ensure_ascii=False, indent=2)

    print(f"✅ サンプルデータを作成しました: {sample_dir}")

    # データ利用可能性を再チェック
    availability = loader.check_data_availability()

📝 サンプルデータを作成中...
✅ サンプルデータを作成しました: data/example/json


In [39]:
# Cell 7: Pydanticモデルとプロンプトビルダー（Structured Outputs対応）
#@title 7. データモデルとプロンプトビルダーの実装 { display-mode: "form" }

from pydantic import BaseModel, Field
from typing import List, Optional, Literal

# Pydanticモデルの定義（Structured Outputs用）
class ExperimentStepOutput(BaseModel):
    """実験手順の1ステップ"""
    id: int = Field(description="ステップ番号")
    text: str = Field(description="実験手順の詳細な説明")

class ExperimentStepsOutput(BaseModel):
    """実験手順全体の出力"""
    steps: List[ExperimentStepOutput] = Field(
        description="実験手順のリスト",
        min_items=1
    )

    # オプション：メタデータ
    total_steps: Optional[int] = Field(
        default=None,
        description="総ステップ数"
    )
    estimated_time: Optional[str] = Field(
        default=None,
        description="推定所要時間"
    )
    safety_notes: Optional[List[str]] = Field(
        default=None,
        description="安全上の注意事項"
    )

# 代替の簡略版モデル（エラー時のフォールバック用）
class SimpleStepsOutput(BaseModel):
    """シンプルな実験手順の出力"""
    steps: List[ExperimentStepOutput]

class PromptBuilder:
    """実験手順生成用のプロンプト構築クラス"""

    def __init__(self, model_type: str = "gpt-4o-mini"):
        self.model_type = model_type

    def build_system_prompt(self) -> str:
        """システムプロンプトの構築"""
        return """あなたは生命科学実験の専門家です。
与えられた実験指示、プロトコル、リソース情報から、実行可能な詳細な実験手順を生成してください。

重要なルール：
1. プロトコルのステップを基に、具体的な実験手順を作成する
2. エイリアス（別名）を適切に解決し、正しい器具名・試薬名を使用する
3. マッピング情報に従って、汎用的な記述を具体的な器具・条件に置き換える
4. 制約条件を満たすように手順を調整する
5. 必要に応じてステップを追加・修正・詳細化する
6. 温度、時間、回転数などの数値は具体的に記載する
7. 安全性と実験の成功に必要な暗黙的な手順も含める
8. 各ステップは明確で実行可能な指示として記述する"""

    def build_task_prompt(self, task: 'ExperimentTask') -> str:
        """タスク用のプロンプトを構築"""

        prompt_parts = []

        # 実験指示
        prompt_parts.append(f"【実験指示】\n{task.instruction}\n")

        # プロトコル
        prompt_parts.append("【参照プロトコル】")
        for step in task.protocol:
            prompt_parts.append(f"{step.id}. {step.text}")
        prompt_parts.append("")

        # リソース情報
        if task.resources.aliases:
            prompt_parts.append("【エイリアス（別名）】")
            for key, value in task.resources.aliases.items():
                prompt_parts.append(f"- {key} = {value}")
            prompt_parts.append("")

        if task.resources.mappings:
            prompt_parts.append("【マッピング（具体化）】")
            for key, value in task.resources.mappings.items():
                prompt_parts.append(f"- {key} → {value}")
            prompt_parts.append("")

        if task.resources.constraints:
            prompt_parts.append("【制約条件】")
            if 'optional_steps' in task.resources.constraints:
                for opt_step in task.resources.constraints['optional_steps']:
                    prompt_parts.append(
                        f"- 「{opt_step['name']}」をステップ{opt_step['earliest_index']}以降に追加"
                    )
            prompt_parts.append("")

        # タスク指示
        prompt_parts.append("【タスク】")
        prompt_parts.append("上記の情報を基に、実行可能な詳細な実験手順を生成してください。")
        prompt_parts.append("以下の点に注意してください：")
        prompt_parts.append("1. エイリアスとマッピングを適用して具体的な器具名を使用")
        prompt_parts.append("2. 実験指示の要件を満たす")
        prompt_parts.append("3. 必要な準備手順（例：機器の電源ON）を追加")
        prompt_parts.append("4. 安全性と成功に必要な詳細を含める")
        prompt_parts.append("5. 制約条件で指定された追加ステップを適切な位置に挿入")

        return "\n".join(prompt_parts)

print("✅ Pydanticモデルとプロンプトビルダーを実装しました")
print(f"   - ExperimentStepsOutput: {len(ExperimentStepsOutput.model_fields)}フィールド")
print(f"   - SimpleStepsOutput: {len(SimpleStepsOutput.model_fields)}フィールド")

✅ Pydanticモデルとプロンプトビルダーを実装しました
   - ExperimentStepsOutput: 4フィールド
   - SimpleStepsOutput: 1フィールド


In [40]:
# Cell 8: GPT実験手順生成器（Structured Outputs専用）
#@title 8. GPT実験手順生成器（Structured Outputs版） { display-mode: "form" }

class GPTExperimentGenerator:
    """GPTを使用した実験手順生成クラス（Structured Outputs専用）"""

    def __init__(self,
                 model: str = "gpt-5-mini-2025-08-07", temperature: float=1.0):
        self.client = OpenAI(api_key=API_KEY)
        self.model = model
        self.temperature = temperature
        self.prompt_builder = PromptBuilder(model_type=model)
        self.max_retries = 3  # パース失敗時の最大リトライ回数

    def generate_steps(self, task: 'ExperimentTask') -> List[Dict]:
        """Structured Outputsを使用して実験手順を生成（リトライ機能付き）"""

        # プロンプトの構築
        system_prompt = self.prompt_builder.build_system_prompt()
        user_prompt = self.prompt_builder.build_task_prompt(task)

        last_error = None

        for attempt in range(self.max_retries):
            try:
                logger.info(f"実験手順生成中... (試行 {attempt + 1}/{self.max_retries})")

                completion = self.client.chat.completions.parse(
                    model=self.model,
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_prompt}
                    ],
                    response_format=ExperimentStepsOutput,
                    temperature=self.temperature
                )

                parsed_output = completion.choices[0].message.parsed

                if parsed_output and parsed_output.steps:
                    steps = [
                        {"id": step.id, "text": step.text}
                        for step in parsed_output.steps
                    ]
                    if parsed_output.total_steps:
                        logger.info(f"総ステップ数: {parsed_output.total_steps}")
                    if parsed_output.estimated_time:
                        logger.info(f"推定時間: {parsed_output.estimated_time}")
                    if parsed_output.safety_notes:
                        logger.info(f"安全注意: {', '.join(parsed_output.safety_notes)}")

                    logger.info(f"✅ 実験手順生成成功 (試行 {attempt + 1})")
                    return steps
                else:
                    logger.warning(f"⚠️ パース失敗 (試行 {attempt + 1}/{self.max_retries}): 出力が空です")
                    last_error = "Empty parsed output"

            except Exception as e:
                logger.warning(f"⚠️ エラー発生 (試行 {attempt + 1}/{self.max_retries}): {str(e)}")
                last_error = e

                if attempt < self.max_retries - 1:
                    wait_time = 2 ** attempt
                    logger.info(f"⏳ {wait_time}秒待機後にリトライします...")
                    time.sleep(wait_time)

        logger.error(f"❌ {self.max_retries}回の試行すべてで実験手順生成に失敗しました")
        logger.error(f"最後のエラー: {last_error}")
        self._log_generation_failure(task, last_error)

        # すべての試行が失敗した場合、プロトコルをそのまま使用
        fallback_steps = [
            {"id": step.id, "text": step.text}
            for step in task.protocol
        ]
        logger.warning(f"⚠️ フォールバック: プロトコルをそのまま使用します（{len(fallback_steps)}ステップ）")

        return fallback_steps

    def apply_resources(self, steps: List[Dict], task: 'ExperimentTask') -> List[Dict]:
        """リソース情報を適用して手順を修正"""

        modified_steps = deepcopy(steps)

        for step in modified_steps:
            text = step.get('text', '')

            # エイリアスの適用
            for alias, actual in task.resources.aliases.items():
                text = text.replace(alias, actual)

            # マッピングの適用
            for generic, specific in task.resources.mappings.items():
                actual_generic = task.resources.aliases.get(generic, generic)
                text = text.replace(actual_generic, specific)

            step['text'] = text

        if 'optional_steps' in task.resources.constraints:
            for opt_step in task.resources.constraints['optional_steps']:
                insert_pos = opt_step['earliest_index']
                new_step = {
                    'id': insert_pos,
                    'text': opt_step['name']
                }

                # 適切な位置に挿入
                if insert_pos <= len(modified_steps):
                    modified_steps.insert(insert_pos - 1, new_step)
                else:
                    modified_steps.append(new_step)

            # IDの再割り当て
            for i, step in enumerate(modified_steps, 1):
                step['id'] = i

        return modified_steps

    def _log_generation_failure(self, task: 'ExperimentTask', error: Any):
        """生成失敗をログファイルに記録"""

        log_dir = Path("./outputs/logs")
        log_dir.mkdir(parents=True, exist_ok=True)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        log_file = log_dir / f"generation_failure_{task.id}_{timestamp}.json"

        log_data = {
            "timestamp": datetime.now().isoformat(),
            "task_id": task.id,
            "instruction": task.instruction,
            "model": self.model,
            "error": str(error),
            "protocol_steps": len(task.protocol),
            "has_ground_truth": task.ground_truth is not None
        }

        with open(log_file, 'w', encoding='utf-8') as f:
            json.dump(log_data, f, ensure_ascii=False, indent=2)

        logger.info(f"📝 エラーログを保存: {log_file}")

print("✅ Structured Outputs専用のGPT実験手順生成器を実装しました")
print("デフォルト設定：")
print("   - モデル: gpt-5-mini-2025-08-07")
print("   - 最大リトライ回数: 3")
print("   - エラーログ: ./outputs/logs/")

✅ Structured Outputs専用のGPT実験手順生成器を実装しました
デフォルト設定：
   - モデル: gpt-5-mini-2025-08-07
   - 最大リトライ回数: 3
   - エラーログ: ./outputs/logs/


In [41]:
# Cell 9: LLMベースの評価システム（5点満点版）
#@title 9. LLMベースの評価システム { display-mode: "form" }

from pydantic import BaseModel, Field
from typing import List, Optional, Dict

# 個別評価項目の結果
class CriterionScore(BaseModel):
    """個別評価項目のスコア"""
    name: str = Field(description="評価項目名")
    score: int = Field(description="スコア（1-5）", ge=1, le=5)
    reasoning: str = Field(description="スコアの根拠")

# 減点項目
class Deduction(BaseModel):
    """減点項目"""
    reason: str = Field(description="減点理由")
    points: float = Field(description="減点数", ge=0)
    examples: List[str] = Field(default_factory=list, description="該当箇所")

# 評価結果
class EvaluationResult(BaseModel):
    """実験手順の評価結果"""
    task_id: str

    # 基本評価（各5点満点）
    completeness: CriterionScore = Field(description="必要な要素がすべて含まれているか")
    specificity: CriterionScore = Field(description="数値や条件が明確か")
    consistency: CriterionScore = Field(description="矛盾がなく論理的か")
    safety: CriterionScore = Field(description="安全上の配慮が適切か")
    feasibility: CriterionScore = Field(description="実際に実行可能か")
    clarity: CriterionScore = Field(description="理解しやすく曖昧さがないか")

    # 減点
    deductions: List[Deduction] = Field(default_factory=list)

    # 総合評価
    raw_score: float = Field(description="減点前の平均スコア")
    final_score: float = Field(description="減点後の最終スコア（1-5）")
    feedback: str = Field(description="総合フィードバック")
    improvements: List[str] = Field(default_factory=list, description="改善提案")

class LLMEvaluator:
    """LLMベースの実験手順評価クラス"""

    def __init__(self,
                 model: str = "gpt-5-mini-2025-08-07",
                 temperature: float = 1.0):
        self.client = OpenAI(api_key=API_KEY)
        self.model = model
        self.temperature = temperature
        self.max_retries = 2

    def evaluate(self,
                task: 'ExperimentTask',
                generated_steps: List[Dict]) -> EvaluationResult:
        """実験手順を評価"""

        system_prompt = self._build_system_prompt()
        user_prompt = self._build_user_prompt(task, generated_steps)

        for attempt in range(self.max_retries):
            try:
                logger.info(f"評価実行中... (試行 {attempt + 1}/{self.max_retries})")

                completion = self.client.chat.completions.parse(
                    model=self.model,
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_prompt}
                    ],
                    response_format=EvaluationResult,
                    temperature=self.temperature
                )

                result = completion.choices[0].message.parsed
                if result:
                    result = self._calculate_final_score(result)
                    logger.info(f"✅ 評価完了: {result.final_score:.2f}/5.0")
                    return result

            except Exception as e:
                logger.warning(f"⚠️ 評価エラー (試行 {attempt + 1}): {str(e)}")
                if attempt < self.max_retries - 1:
                    time.sleep(2)

        logger.error("❌ 評価に失敗しました")
        return self._create_default_evaluation(task.id, len(generated_steps))

    def _build_system_prompt(self) -> str:
        """評価用システムプロンプト"""
        return """あなたは生命科学実験の専門家です。実験手順を以下の基準で評価してください。

【採点基準（5点満点）】
1点: 誤っている／要件を満たしていない
2点: 誤っているが、方向性は合っている
3点: 部分的に正しい
4点: 正しい／要件を満たしている
5点: 優れている／非常に実用的

【評価項目】
1. 完全性: 必要な器具、試薬、手順がすべて含まれているか
2. 具体性: 温度、時間、量などが具体的な数値で示されているか
3. 一貫性: 手順間で矛盾がなく論理的か
4. 安全性: 安全上の配慮が適切か
5. 実行可能性: 実際のラボで実行可能か
6. 明確性: 指示が明確で理解しやすいか

【減点項目】
- 不自然な日本語: -1点
- 事実と異なる記述: -1点
- 過度な安全性: 2点に固定
- 単位の欠落: -0.5点
- 曖昧な表現: -0.5点

各項目を採点し、根拠を説明してください。
減点項目があれば具体例とともに記載してください。"""

    def _build_user_prompt(self,
                          task: 'ExperimentTask',
                          generated_steps: List[Dict]) -> str:
        """評価用ユーザープロンプト"""

        prompt_parts = []

        prompt_parts.append(f"【タスク】{task.instruction}\n")

        prompt_parts.append("【生成された手順】")
        for step in generated_steps:
            prompt_parts.append(f"{step['id']}. {step['text']}")
        prompt_parts.append("")

        prompt_parts.append("【元のプロトコル】")
        for step in task.protocol:
            prompt_parts.append(f"{step.id}. {step.text}")
        prompt_parts.append("")

        if task.resources.aliases or task.resources.mappings:
            prompt_parts.append("【リソース情報】")
            if task.resources.aliases:
                for k, v in task.resources.aliases.items():
                    prompt_parts.append(f"  {k} = {v}")
            if task.resources.mappings:
                for k, v in task.resources.mappings.items():
                    prompt_parts.append(f"  {k} → {v}")
            prompt_parts.append("")

        prompt_parts.append("上記の実験手順を6つの項目で評価し、減点項目をチェックしてください。")

        return "\n".join(prompt_parts)

    def _calculate_final_score(self, result: EvaluationResult) -> EvaluationResult:
        """最終スコアを計算"""

        # 基本スコアの平均
        scores = [
            result.completeness.score,
            result.specificity.score,
            result.consistency.score,
            result.safety.score,
            result.feasibility.score,
            result.clarity.score
        ]
        result.raw_score = sum(scores) / len(scores)

        # 減点適用
        total_deduction = 0
        for deduction in result.deductions:
            if "過度な安全性" in deduction.reason:
                result.final_score = 2.0
                return result
            total_deduction += deduction.points

        result.final_score = max(1.0, result.raw_score - total_deduction)
        return result

    def _create_default_evaluation(self, task_id: str, num_steps: int) -> EvaluationResult:
        """デフォルト評価"""

        default = CriterionScore(
            name="default",
            score=3,
            reasoning="評価失敗のため中間値を使用"
        )

        return EvaluationResult(
            task_id=task_id,
            completeness=default,
            specificity=default,
            consistency=default,
            safety=default,
            feasibility=default,
            clarity=default,
            deductions=[],
            raw_score=3.0,
            final_score=3.0,
            feedback="評価を完了できませんでした",
            improvements=[]
        )

    def create_summary(self, results: List[EvaluationResult]) -> pd.DataFrame:
        """評価結果のサマリー作成"""

        data = []
        for r in results:
            data.append({
                'task_id': r.task_id,
                'completeness': r.completeness.score,
                'specificity': r.specificity.score,
                'consistency': r.consistency.score,
                'safety': r.safety.score,
                'feasibility': r.feasibility.score,
                'clarity': r.clarity.score,
                'deductions': len(r.deductions),
                'final_score': r.final_score
            })

        return pd.DataFrame(data)

print("✅ 評価システムを実装しました")
print("  - 5点満点評価")
print("  - 6つの評価項目")
print("  - 自動減点機能")

✅ 評価システムを実装しました
  - 5点満点評価
  - 6つの評価項目
  - 自動減点機能


In [42]:
# Cell 10: メインパイプライン（評価システム統合版）
#@title 10. メインパイプライン { display-mode: "form" }

import uuid
from datetime import datetime
import shutil


class RunManager:
    """実行ごとの結果管理クラス"""

    def __init__(self, base_dir: str = "./outputs"):
        self.base_dir = Path(base_dir)
        self.runs_dir = self.base_dir / "runs"
        self.runs_dir.mkdir(parents=True, exist_ok=True)

        # 実行IDの生成（タイムスタンプ + 短いUUID）
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        short_id = str(uuid.uuid4())[:4]
        self.run_id = f"{timestamp}_{short_id}"

        self.run_dir = self.runs_dir / self.run_id
        self.run_dir.mkdir(parents=True, exist_ok=True)

        (self.run_dir / "evaluations").mkdir(exist_ok=True)
        (self.run_dir / "logs").mkdir(exist_ok=True)

        logger.info(f"📁 実行ID: {self.run_id}")
        logger.info(f"📁 結果保存先: {self.run_dir}")

        # 実行履歴の更新
        self._update_run_history()

    def save_config(self, config: Dict):
        config_file = self.run_dir / "config.json"
        config_data = {
            "run_id": self.run_id,
            "timestamp": datetime.now().isoformat(),
            **config
        }
        with open(config_file, 'w', encoding='utf-8') as f:
            json.dump(config_data, f, ensure_ascii=False, indent=2)

    def get_path(self, filename: str) -> Path:
        return self.run_dir / filename

    def update_latest(self):
        """最新実行フォルダを更新"""
        latest_dir = self.base_dir / "latest"

        # 既存のlatestディレクトリを削除
        if latest_dir.exists():
            shutil.rmtree(latest_dir)

        # 現在の実行内容をlatestにコピー
        shutil.copytree(self.run_dir, latest_dir)

        logger.info(f"📁 最新結果を更新: {latest_dir}")

    def _update_run_history(self):
        """実行履歴を更新"""
        history_file = self.base_dir / "run_history.json"

        # 既存の履歴を読み込み
        if history_file.exists():
            with open(history_file, 'r', encoding='utf-8') as f:
                history = json.load(f)
        else:
            history = {"runs": []}

        # 新しい実行を追加
        history["runs"].append({
            "run_id": self.run_id,
            "start_time": datetime.now().isoformat(),
            "status": "running"
        })

        # 保存
        with open(history_file, 'w', encoding='utf-8') as f:
            json.dump(history, f, ensure_ascii=False, indent=2)

    def finalize(self, status: str = "completed", summary: Dict = None):
        """実行を完了"""
        history_file = self.base_dir / "run_history.json"

        if history_file.exists():
            with open(history_file, 'r', encoding='utf-8') as f:
                history = json.load(f)

            # 該当する実行を更新
            for run in history["runs"]:
                if run["run_id"] == self.run_id:
                    run["status"] = status
                    run["end_time"] = datetime.now().isoformat()
                    if summary:
                        run["summary"] = summary
                    break

            with open(history_file, 'w', encoding='utf-8') as f:
                json.dump(history, f, ensure_ascii=False, indent=2)

        # 最新フォルダを更新
        self.update_latest()

        logger.info(f"✅ 実行完了: {self.run_id} ({status})")


class ExperimentPipeline:
    """実験手順生成と評価のメインパイプライン"""

    def __init__(self, model: str = "gpt-5-mini-2025-08-07", temperature: float= 1.0):
        if not API_KEY:
            raise ValueError("APIキーが設定されていません")
        self.gpt_generator = GPTExperimentGenerator(model=model, temperature=temperature)
        self.llm_evaluator = LLMEvaluator(model=model, temperature=temperature)
        self.data_loader = DataLoader()
        self.run_manager = RunManager()

        # 設定を保存
        self.run_manager.save_config({
            "model": model,
            "temperature": temperature,
            "generator": "structured_outputs",
            "evaluator": "llm_5point"
        })

        logger.info(f"✅ Pipeline initialized with model: {model}, temperature: {temperature}")

    def process_task(self, task: 'ExperimentTask') -> Dict:
        """単一タスクの処理（生成＋評価）"""

        logger.info(f"🔬 Processing task {task.id}: {task.instruction[:50]}...")
        # 手順生成
        generated_steps = self.gpt_generator.generate_steps(task)
        # リソース適用
        generated_steps = self.gpt_generator.apply_resources(generated_steps, task)
        # LLM評価
        evaluation = self.llm_evaluator.evaluate(task, generated_steps)

        logger.info(f"📊 評価スコア: {evaluation.final_score:.2f}/5.0")

        result = {
            'task_id': task.id,
            'instruction': task.instruction,
            'generated_steps': generated_steps,
            'evaluation': evaluation,
            'timestamp': datetime.now().isoformat()
        }

        return result

    def process_dataset(self,
                       dataset_type: str = "example",
                       save_results: bool = True) -> Tuple[pd.DataFrame, List[Dict]]:
        """データセット全体の処理"""

        tasks = self.data_loader.load_dataset(dataset_type)
        if not tasks:
            logger.error(f"❌ No tasks found in {dataset_type} dataset")
            self.run_manager.finalize(
                status="failed",
                summary={"error": "No tasks found"})
            return pd.DataFrame(), []

        logger.info(f"📚 Processing {len(tasks)} tasks from {dataset_type} dataset")

        self.run_manager.save_config({
            "dataset_type": dataset_type,
            "num_tasks": len(tasks)
        })

        results = []
        evaluations = []

        # 各タスクを処理
        for task in tqdm(tasks, desc=f"Processing {dataset_type} dataset"):
            try:
                result = self.process_task(task)
                results.append(result)
                evaluations.append(result['evaluation'])

                # 個別結果を即座に保存
                if save_results:
                    eval_file = self.run_manager.get_path(f"evaluations/{task.id}.json")
                    with open(eval_file, 'w', encoding='utf-8') as f:
                        json.dump({
                            'task_id': result['task_id'],
                            'instruction': result['instruction'],
                            'num_steps': len(result['generated_steps']),
                            'evaluation': result['evaluation'].model_dump()
                        }, f, ensure_ascii=False, indent=2)

            except Exception as e:
                logger.error(f"Task {task.id} failed: {e}")
                # エラーもログに記録
                error_file = self.run_manager.get_path(f"logs/error_{task.id}.txt")
                with open(error_file, 'w') as f:
                    f.write(str(e))

            # レート制限対策
            time.sleep(1)

        # 結果の保存
        if save_results:
            self._save_results(results, dataset_type)

        # 評価サマリーの作成
        summary_df = self.llm_evaluator.create_summary(evaluations)

        # サマリーをCSVで保存
        if not summary_df.empty:
            summary_file = self.run_manager.get_path("summary.csv")
            summary_df.to_csv(summary_file, index=False)

            # 実行完了
            self.run_manager.finalize(status="completed", summary={
                "num_tasks": len(tasks),
                "num_completed": len(results),
                "avg_score": summary_df['final_score'].mean(),
                "dataset_type": dataset_type
            })

        return summary_df, results

    def _save_results(self, results: List[Dict], dataset_type: str):
        """結果の保存"""

        # リーダーボード提出形式
        submission = {
            'team_name': 'baseline',
            'submission_time': datetime.now().isoformat(),
            'dataset_type': dataset_type,
            'model': self.gpt_generator.model,
            'predictions': []
        }

        for result in results:
            submission['predictions'].append({
                'task_id': result['task_id'],
                'steps': result['generated_steps']
            })

        submission_file = self.run_manager.get_path("submission.json")
        with open(submission_file, 'w', encoding='utf-8') as f:
            json.dump(submission, f, ensure_ascii=False, indent=2)

        logger.info(f"💾 Submission saved to {submission_file}")

print("✅ 実行管理機能付きメインパイプラインを実装しました")
print("特徴:")
print("  - 実行ごとに一意のフォルダ（YYYYMMDD_HHMMSS_ID）")
print("  - 実行設定の自動保存")
print("  - 最新実行へのアクセス（latest/）")
print("  - 実行履歴の管理（run_history.json）")

✅ 実行管理機能付きメインパイプラインを実装しました
特徴:
  - 実行ごとに一意のフォルダ（YYYYMMDD_HHMMSS_ID）
  - 実行設定の自動保存
  - 最新実行へのアクセス（latest/）
  - 実行履歴の管理（run_history.json）


In [43]:
# Cell 11: 実行
#@title 11. ベースライン実行 { run: "auto" }
#@markdown 以下の設定でベースライン手法を実行します

dataset_type = "example" #@param ["example", "public", "private"]
save_results = True #@param {type:"boolean"}
model_name = "gpt-5-mini-2025-08-07" #@param ["gpt-4.1-mini-2025-04-14", "gpt-4o-2024-08-06", "gpt-5-2025-08-07", "gpt-5-mini-2025-08-07", "gpt-5-nano-2025-08-07"]
#@markdown gpt-4o-mini, gpt-4o-2024-08-06, あるいはそれ以降のモデルに対応しています。<br>
#@markdown (Structured outputを使用しているため) <br>
#@markdown gpt-5系モデルを使用する場合、temperature=1.0としてください。
temperature = 1.0 #@param

if not API_KEY:
    print("❌ APIキーが設定されていません")
    print("Cell 2でAPIキーを設定してください")

else:
    print("="*60)
    print("🚀 LA-Bench 2025 Baseline Execution")
    print("="*60)
    print(f"Dataset: {dataset_type}")
    print(f"Model: {model_name}")
    print(f"Temperature: {temperature}")
    print("="*60)

    # パイプラインの初期化と実行
    pipeline = ExperimentPipeline(model=model_name, temperature=temperature)
    summary_df, results = pipeline.process_dataset(
        dataset_type=dataset_type,
        save_results=save_results
    )

    # 結果の表示
    if not summary_df.empty:
        print("\n📊 評価結果サマリー（5点満点）")
        print("="*60)

        # 各タスクの評価
        print("\n【タスク別評価】")
        display_cols = ['task_id', 'final_score', 'completeness', 'specificity',
                        'consistency', 'safety', 'feasibility', 'clarity']
        print(summary_df[display_cols].to_string(index=False))

        # 統計情報
        print("\n【統計情報】")
        print(f"タスク数: {len(summary_df)}")
        print(f"平均スコア: {summary_df['final_score'].mean():.2f}/5.0")
        print(f"最高スコア: {summary_df['final_score'].max():.2f}/5.0")
        print(f"最低スコア: {summary_df['final_score'].min():.2f}/5.0")
        print(f"標準偏差: {summary_df['final_score'].std():.2f}")

        # 評価項目別の平均
        print("\n【評価項目別平均スコア】")
        criteria = ['completeness', 'specificity', 'consistency',
                    'safety', 'feasibility', 'clarity']
        for criterion in criteria:
            avg_score = summary_df[criterion].mean()
            print(f"  {criterion:12s}: {avg_score:.2f}/5.0")

        # 減点の統計
        if 'deductions' in summary_df.columns:
            total_deductions = summary_df['deductions'].sum()
            if total_deductions > 0:
                print(f"\n【減点統計】")
                print(f"  総減点項目数: {total_deductions}")
                print(f"  減点があったタスク: {(summary_df['deductions'] > 0).sum()}/{len(summary_df)}")

🚀 LA-Bench 2025 Baseline Execution
Dataset: example
Model: gpt-5-mini-2025-08-07
Temperature: 1.0


Processing example dataset:   0%|          | 0/2 [00:00<?, ?it/s]


📊 評価結果サマリー（5点満点）

【タスク別評価】
                  task_id  final_score  completeness  specificity  consistency  safety  feasibility  clarity
T75_fibroblast_thawing_v1     2.000000             4            3            3       4            4        3
                 task_001     2.333333             4            4            3       5            4        3

【統計情報】
タスク数: 2
平均スコア: 2.17/5.0
最高スコア: 2.33/5.0
最低スコア: 2.00/5.0
標準偏差: 0.24

【評価項目別平均スコア】
  completeness: 4.00/5.0
  specificity : 3.50/5.0
  consistency : 3.00/5.0
  safety      : 4.50/5.0
  feasibility : 4.00/5.0
  clarity     : 3.00/5.0

【減点統計】
  総減点項目数: 4
  減点があったタスク: 2/2


In [54]:
# Cell 12: 結果の可視化と分析
#@title 12. 結果の可視化と分析 { display-mode: "form" }

import numpy as np
import json
from pathlib import Path

if 'summary_df' in locals() and not summary_df.empty:
    print("="*60)
    print("📊 LA-Bench 2025 評価結果分析")
    print("="*60)

    # 1. 全体パフォーマンス
    print("\n【全体パフォーマンス】")
    mean_score = summary_df['final_score'].mean()
    std_score = summary_df['final_score'].std()
    ci_95 = 1.96 * std_score / np.sqrt(len(summary_df))

    print(f"  最終スコア: {mean_score:.2f} ± {ci_95:.2f} (95% CI)")
    print(f"  中央値: {summary_df['final_score'].median():.2f}")
    print(f"  四分位範囲: [{summary_df['final_score'].quantile(0.25):.2f}, {summary_df['final_score'].quantile(0.75):.2f}]")

    # パフォーマンス判定
    if mean_score >= 4.0:
        grade = "🏆 優秀"
    elif mean_score >= 3.5:
        grade = "✅ 良好"
    elif mean_score >= 3.0:
        grade = "📈 改善余地あり"
    else:
        grade = "⚠️ 要改善"
    print(f"  判定: {grade}")

    # 2. スコア分布の詳細分析
    print("\n【スコア分布】")
    hist_data = []
    for i in range(1, 6):
        if i == 5:
            count = (summary_df['final_score'] == i).sum()
        else:
            count = ((summary_df['final_score'] >= i) & (summary_df['final_score'] < i+1)).sum()
        pct = count / len(summary_df) * 100
        bar = "█" * int(pct / 2)
        hist_data.append(f"  [{i}]: {count:3d} ({pct:5.1f}%) {bar}")
    for line in hist_data:
        print(line)

    # 3. 評価項目の強み・弱み分析
    criteria = ['completeness', 'specificity', 'consistency',
               'safety', 'feasibility', 'clarity']
    available = [c for c in criteria if c in summary_df.columns]

    if available:
        print("\n【評価項目分析】")
        criteria_stats = []
        for c in available:
            mean = summary_df[c].mean()
            std = summary_df[c].std()
            criteria_stats.append((c, mean, std))

        # ソートして表示
        criteria_stats.sort(key=lambda x: x[1], reverse=True)

        print("  強み（上位3項目）:")
        for c, mean, std in criteria_stats[:3]:
            indicator = "◆" * int(mean)
            print(f"    {c:12s}: {mean:.2f}±{std:.2f} {indicator}")

        print("  弱み（下位3項目）:")
        for c, mean, std in criteria_stats[-3:]:
            indicator = "◇" * int(5 - mean)
            print(f"    {c:12s}: {mean:.2f}±{std:.2f} {indicator}")

        # 改善優先度（標準偏差が大きく平均が低い）
        priority = sorted(criteria_stats, key=lambda x: -x[2] * (5 - x[1]))
        print(f"  最優先改善項目: {priority[0][0]} (不安定かつ低スコア)")

    # 4. 減点パターン分析
    if 'deductions' in summary_df.columns:
        total_deductions = summary_df['deductions'].sum()
        if total_deductions > 0:
            print("\n【減点分析】")
            no_deduction = (summary_df['deductions'] == 0).sum()
            with_deduction = (summary_df['deductions'] > 0).sum()
            print(f"  減点なし: {no_deduction} ({no_deduction/len(summary_df)*100:.1f}%)")
            print(f"  減点あり: {with_deduction} ({with_deduction/len(summary_df)*100:.1f}%)")

            # 減点の影響度
            if with_deduction > 0:
                impact = (summary_df[summary_df['deductions'] == 0]['final_score'].mean() -
                         summary_df[summary_df['deductions'] > 0]['final_score'].mean())
                print(f"  減点による平均スコア低下: -{impact:.2f}")

    # 5. 外れ値とリスクタスク
    print("\n【要注意タスク】")
    threshold = mean_score - 2 * std_score
    outliers = summary_df[summary_df['final_score'] < threshold]
    if len(outliers) > 0:
        print(f"  外れ値タスク（< {threshold:.2f}）: {len(outliers)}件")
        for _, row in outliers.head(3).iterrows():
            print(f"    - {row['task_id']}: {row['final_score']:.2f}")
    else:
        print("  外れ値なし（安定した性能）")

    # 失敗タスク（スコア < 3.0）
    failed = summary_df[summary_df['final_score'] < 3.0]
    if len(failed) > 0:
        print(f"  失敗タスク（< 3.0）: {len(failed)}件")

    # 6. 改善ポテンシャル
    print("\n【改善ポテンシャル】")
    perfect_gap = 5.0 - mean_score
    achievable_target = summary_df['final_score'].quantile(0.75)
    improvement_potential = achievable_target - mean_score

    print(f"  理論的改善余地: {perfect_gap:.2f}ポイント")
    print(f"  現実的改善目標: {achievable_target:.2f} (+{improvement_potential:.2f})")

    if available:
        # 最も改善効果が高い項目
        impacts = []
        for c in available:
            potential = (5.0 - summary_df[c].mean()) * (1.0 / len(available))
            impacts.append((c, potential))
        best_impact = max(impacts, key=lambda x: x[1])
        print(f"  最大改善効果: {best_impact[0]} (+{best_impact[1]:.2f}ポイント期待)")

    print("\n" + "="*60)

    # 7. 結果の保存
    try:
        # 保存先の決定
        if 'pipeline' in locals() and hasattr(pipeline, 'run_manager'):
            base_path = pipeline.run_manager.run_dir
            run_id = pipeline.run_manager.run_id
        else:
            base_path = Path("./outputs/baseline")
            base_path.mkdir(parents=True, exist_ok=True)
            run_id = "baseline"

        # 詳細CSV
        csv_file = base_path / "evaluation_results.csv"
        summary_df.to_csv(csv_file, index=False)

        # 分析サマリJSON
        analysis = {
            'run_id': run_id,
            'timestamp': str(datetime.now()),
            'performance': {
                'mean': float(mean_score),
                'std': float(std_score),
                'ci_95': float(ci_95),
                'median': float(summary_df['final_score'].median()),
                'q1': float(summary_df['final_score'].quantile(0.25)),
                'q3': float(summary_df['final_score'].quantile(0.75))
            },
            'criteria': {c: {
                'mean': float(summary_df[c].mean()),
                'std': float(summary_df[c].std())
            } for c in available},
            'summary': {
                'num_tasks': len(summary_df),
                'num_failed': len(failed),
                'num_outliers': len(outliers),
                'improvement_potential': float(improvement_potential)
            }
        }

        json_file = base_path / "analysis_summary.json"
        with open(json_file, 'w') as f:
            json.dump(analysis, f, indent=2)

        print(f"💾 結果を保存しました:")
        print(f"  - {csv_file}")
        print(f"  - {json_file}")

    except Exception as e:
        print(f"⚠️ 保存エラー: {e}")

    # Colab用の詳細表示
    if IN_COLAB:
        from IPython.display import display
        print("\n【スコア要約統計】")
        display(summary_df['final_score'].describe())

else:
    print("⚠️ データがありません。Cell 11を実行してください。")

📊 LA-Bench 2025 評価結果分析

【全体パフォーマンス】
  最終スコア: 2.17 ± 0.33 (95% CI)
  中央値: 2.17
  四分位範囲: [2.08, 2.25]
  判定: ⚠️ 要改善

【スコア分布】
  [1]:   0 (  0.0%) 
  [2]:   2 (100.0%) ██████████████████████████████████████████████████
  [3]:   0 (  0.0%) 
  [4]:   0 (  0.0%) 
  [5]:   0 (  0.0%) 

【評価項目分析】
  強み（上位3項目）:
    safety      : 4.50±0.71 ◆◆◆◆
    completeness: 4.00±0.00 ◆◆◆◆
    feasibility : 4.00±0.00 ◆◆◆◆
  弱み（下位3項目）:
    specificity : 3.50±0.71 ◇
    consistency : 3.00±0.00 ◇◇
    clarity     : 3.00±0.00 ◇◇
  最優先改善項目: specificity (不安定かつ低スコア)

【減点分析】
  減点なし: 0 (0.0%)
  減点あり: 2 (100.0%)
  減点による平均スコア低下: -nan

【要注意タスク】
  外れ値なし（安定した性能）
  失敗タスク（< 3.0）: 2件

【改善ポテンシャル】
  理論的改善余地: 2.83ポイント
  現実的改善目標: 2.25 (+0.08)
  最大改善効果: consistency (+0.33ポイント期待)

💾 結果を保存しました:
  - outputs/runs/20250810_093739_1729/evaluation_results.csv
  - outputs/runs/20250810_093739_1729/analysis_summary.json

【スコア要約統計】


,final_score
count,2.000000
mean,2.166667
std,0.235702
min,2.000000
25%,2.083333
50%,2.166667
75%,2.250000
max,2.333333


In [56]:
# Cell 13: 詳細な評価結果の表示
#@title 13. 詳細な評価結果の表示 { display-mode: "form" }

task_index = 0 #@param {type:"slider", min:0, max:10, step:1}

if 'results' in locals() and results and task_index < len(results):
    result = results[task_index]
    eval_result = result['evaluation']

    print("="*60)
    print(f"📋 Task {result['task_id']}: 詳細評価結果")
    print("="*60)

    print(f"\n📝 実験指示:")
    print(f"  {result['instruction']}")

    print(f"\n📊 総合評価:")
    print(f"  最終スコア: {eval_result.final_score:.2f}/5.0")
    print(f"  （減点前: {eval_result.raw_score:.2f}/5.0）")

    print(f"\n📈 評価項目別スコア:")
    for criterion in ['completeness', 'specificity', 'consistency',
                     'safety', 'feasibility', 'clarity']:
        score_obj = getattr(eval_result, criterion)
        print(f"  {criterion:12s}: {score_obj.score}/5 - {score_obj.reasoning[:50]}...")

    if eval_result.deductions:
        print(f"\n⚠️ 減点項目:")
        for ded in eval_result.deductions:
            print(f"  - {ded.reason} (-{ded.points}点)")
            if ded.examples:
                print(f"    例: {ded.examples[0][:50]}...")

    print(f"\n💬 総合フィードバック:")
    print(f"  {eval_result.feedback}")

    if eval_result.improvements:
        print(f"\n💡 改善提案:")
        for imp in eval_result.improvements[:3]:  # 最初の3つ
            print(f"  - {imp}")

    print(f"\n🔬 生成された手順（{len(result['generated_steps'])}ステップ）:")
    for step in result['generated_steps'][:5]:  # 最初の5ステップ
        print(f"  {step['id']}. {step['text'][:80]}...")
    if len(result['generated_steps']) > 5:
        print(f"  ... 他 {len(result['generated_steps']) - 5} ステップ")
else:
    print("⚠️ 表示するデータがありません")

📋 Task 0002: 詳細評価結果

📝 実験指示:
  24ウェルプレートで細胞の培地交換を行ってください

📊 総合評価:
  最終スコア: 2.33/5.0
  （減点前: 3.83/5.0）

📈 評価項目別スコア:
  completeness: 4/5 - PPE、BSC、滅菌ピペット、アスピレーター、温めた培地、廃棄手順、インキュベーター戻しまで一連の要...
  specificity : 4/5 - 温度（37°C）、温め時間（15–30分）、添加量（1.0 mL/ウェル）、揺す回数（3–5回）など...
  consistency : 3/5 - 手順全体は論理的だが、文中に繰り返し（例："DMEMDMEM"、"24ウェル24ウェル"）があり読み...
  safety      : 5/5 - PPE着用、BSC内作業、廃液の10%次亜塩素酸ナトリウムでの処理、作業面のエタノール拭き、汚染物の...
  feasibility : 4/5 - 一般的な細胞培養ラボでそのまま実行可能な手順である。ただし、吸引やピペッティング時の細かな取り扱い指...
  clarity     : 3/5 - 大部分はわかりやすいが、語句の重複（DMEMDMEM 等）やいくつかの曖昧な表現（「速やかに」「必要...

⚠️ 減点項目:
  - 不自然な日本語（語句の重複・繰り返し）: 例）"DMEMDMEM"、"24ウェル24ウェル" といった重複表現。 (-1.0点)
  - 曖昧な表現（手順内の矛盾）: 例）ステップ7の「完全に除去する（残量は残さないが、ウェル底面に触れない）」は実務上の意味が曖昧。 (-0.5点)

💬 総合フィードバック:
  全体として培地交換の標準手順をよくカバーしており、安全面の配慮も十分で実用的です。ただし文書に繰り返しや矛盾が散見され、初心者がそのまま読むと実施時に迷う箇所があります。特に“完全に除去する”という表現の解釈（残量を残すのか否か）や、吸引位置・速度などの定量的指示の欠如を改善してください。また日本語の重複を修正すると読みやすくなります。

💡 改善提案:
  - 文書内の重複表現（例："DMEMDMEM"、"24ウェル24ウェル"）を削除して自然な日本語に統一する。
  - ステップ7の記述を明確化する。例：